# XRT KB Mirror Demo

For ophyd beamline setup see: 
- https://github.com/NSLS-II/blop/blob/main/src/blop/sim/xrt_beamline.py
- https://github.com/NSLS-II/blop/blob/main/src/blop/sim/xrt_kb_model.py

The picture below displays beam from geometric source propagating through a pair of toroidal mirrors focusing the beam on screen. Simulation of a KB setup.

![xrt_blop_layout_w.jpg](../_static/xrt_blop_layout_w.jpg)

In [1]:
import time
from datetime import datetime

import plotly.io as pio

pio.renderers.default = "notebook"
import bluesky.plan_stubs as bps  # noqa F401
import bluesky.plans as bp  # noqa F401
import databroker  # type: ignore[import-untyped]
import matplotlib.pyplot as plt
import tiled.client.container
from bluesky.callbacks import best_effort
from bluesky.callbacks.tiled_writer import TiledWriter
from bluesky.run_engine import RunEngine
from databroker import Broker
from ophyd.utils import make_dir_tree  # type: ignore[import-untyped]
from tiled.client import from_uri  # type: ignore[import-untyped]
from tiled.server import SimpleTiledServer

from blop import DOF, Objective
from blop.ax import Agent
from blop.sim import HDF5Handler
from blop.sim.xrt_beamline import DatabrokerBeamline, TiledBeamline

DETECTOR_STORAGE = "/tmp/blop/sim"

/home/thopkins1/Documents/public-repos/blop/.pixi/envs/dev/lib/python3.12/site-packages/databroker/assets/base_registry.py:14: UserWarning:

pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.

[WARNING 09-25 16:43:47] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [2]:
tiled_server = SimpleTiledServer(readable_storage=[DETECTOR_STORAGE])
tiled_client = from_uri(tiled_server.uri)
tiled_writer = TiledWriter(tiled_client)


def setup_re_env(db_type="default", root_dir="/default/path", method="tiled"):
    RE = RunEngine({})
    bec = best_effort.BestEffortCallback()
    RE.subscribe(bec)
    _ = make_dir_tree(datetime.now().year, base_path=root_dir)

    if method == "tiled":
        RE.subscribe(tiled_writer)
        return {"RE": RE, "db": tiled_client, "bec": bec}

    elif method == "databroker":
        db = Broker.named(db_type)
        db.reg.register_handler("HDF5", HDF5Handler, overwrite=True)
        try:
            databroker.assets.utils.install_sentinels(db.reg.config, version=1)
        except Exception:
            pass
        RE.subscribe(db.insert)
        return {
            "RE": RE,
            "db": db,
            "bec": bec,
        }
    else:
        raise ValueError("The method for data storage used is not supported")


def register_handlers(db, handlers):
    for handler_spec, handler_class in handlers.items():
        db.reg.register_handler(handler_spec, handler_class, overwrite=True)


env = setup_re_env(db_type="temp", root_dir="/tmp/blop/sim", method="tiled")
globals().update(env)
bec.disable_plots()

2025-09-25 16:43:48.876 INFO: Subprocess stdout: 
2025-09-25 16:43:48.876 INFO: Subprocess stderr: Database sqlite+aiosqlite:////tmp/tmpu03rcrhb/catalog.db is new. Creating tables.
Database initialized.

Tiled version 0.1.0b33
2025-09-25 16:43:49.012 INFO: Tiled version 0.1.0b33
2025-09-25 16:43:49.015 INFO: Context impl SQLiteImpl.
2025-09-25 16:43:49.015 INFO: Will assume non-transactional DDL.
2025-09-25 16:43:49.145 INFO: HTTP Request: GET http://127.0.0.1:44993/api/v1?api_key=d087bdbdb796ec81 "HTTP/1.1 307 Temporary Redirect"
2025-09-25 16:43:49.153 INFO: HTTP Request: GET http://127.0.0.1:44993/api/v1/ "HTTP/1.1 200 OK"
2025-09-25 16:43:49.180 INFO: HTTP Request: GET http://127.0.0.1:44993/api/v1/metadata/ "HTTP/1.1 200 OK"
2025-09-25 16:43:49.186 INFO: HTTP Request: GET http://127.0.0.1:44993/api/v1/metadata/?include_data_sources=true "HTTP/1.1 200 OK"


In [3]:
plt.ion()

h_opt = 0
dh = 5

R1, dR1 = 40000, 10000
R2, dR2 = 20000, 10000

In [4]:
if isinstance(db, tiled.client.container.Container):
    beamline = TiledBeamline(name="bl")
else:
    beamline = DatabrokerBeamline(name="bl")
time.sleep(1)
dofs = [
    DOF(movable=beamline.kbv_dsv, search_domain=(R1 - dR1, R1 + dR1)),
    DOF(movable=beamline.kbh_dsh, search_domain=(R2 - dR2, R2 + dR2)),
]

In [5]:
objectives = [
    Objective(name="bl_det_sum", target="max", constraint=(20, 1e12)),
    Objective(
        name="bl_det_wid_x",
        target="min",
    ),
    Objective(
        name="bl_det_wid_y",
        target="min",
    ),
]

In [6]:
agent = Agent(
    readables=[beamline.det],
    dofs=dofs,
    objectives=objectives,
    db=db,
)
agent.configure_experiment(
    name="xrt-blop-demo",
    description="A demo of the Blop agent with XRT simulated beamline",
    experiment_type="demo",
)

2025-09-25 16:43:50.206 INFO: Configuring optimization with objective: bl_det_sum, -bl_det_wid_x, -bl_det_wid_y and outcome constraints: ['bl_det_sum >= 20', 'bl_det_sum <= 1000000000000.0']


UserInputError: More than one objective threshold specified for metric bl_det_sum.

In [ ]:
RE(agent.learn(iterations=25))

In [ ]:
_ = agent.plot_objective(x_dof_name="bl_kbh_dsh", y_dof_name="bl_kbv_dsv", objective_name="bl_det_sum")

## Visualizing the optimal beam

Below we get the optimal parameters, move the motors to their optimal positions, and observe the resulting beam.

In [ ]:
optimal_parameters = next(iter(agent.client.get_pareto_frontier()))[0]
optimal_parameters

In [ ]:
from bluesky.plans import list_scan

scan_motor_params = []
for motor in [beamline.kbv_dsv, beamline.kbh_dsh]:
    scan_motor_params.append(motor)
    scan_motor_params.append([optimal_parameters[motor.name]])
uid = RE(list_scan([beamline.det], *scan_motor_params))

In [ ]:
import matplotlib.pyplot as plt

image = db[uid[0]]["streams"]["primary"]["bl_det_image"].read().squeeze()
plt.imshow(image)
plt.colorbar()
plt.show()